### This is the Week 3 assignment - Segmenting and Clustering Neighborhoods in Toronto

#### Install BeautifulSoup

In [1]:
!conda install beautifulsoup4 --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.6.3       |           py36_0         138 KB
    python-3.6.7               |       h0371630_0        34.3 MB
    cryptography-2.4.2         |   py36h1ba5d50_0         618 KB
    grpcio-1.16.1              |   py36hf8bcb03_1         1.1 MB
    libarchive-3.3.3           |       h5d8350f_5         1.5 MB
    certifi-2018.11.29         |           py36_0         146 KB
    ------------------------------------------------------------
                                           Total:        37.7 MB

The following packages will be UPDATED:

    beautifulsoup4:  4.6.3-py36_0            anaconda    --> 4.6.3-py36_0           
    cryptography:    2.3.1-py36hdffb7b8_0    conda-forge --> 2.4.

#### Install other libraries mentioned in the video

In [3]:
!conda install lxml --yes
!conda install html5lib --yes
!conda install requests --yes

Solving environment: done

# All requested packages already installed.

Solving environment: - 
  - defaults::html5lib-0.9999999-py36_0
  - anaconda::html5lib-0.9999999-py36done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - html5lib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    html5lib-0.9999999         |           py36_0         176 KB

The following packages will be UPDATED:

    html5lib: 0.9999999-py36_0 anaconda --> 0.9999999-py36_0


html5lib-0.9999999   | 176 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - requests


The following packages will be downloaded:

    package                    |            build

#### Do basic imports

In [128]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

#### Open file and transform it on a soup object (bs4.BeautifulSoup)

In [2]:
response = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(response.text,'lxml')

In [3]:
myTable = soup.find('table')

##### Now that we have the table, let's loop through the rows and create a list for each cell

In [69]:
table_list = []

for row in myTable.tbody.find_all('tr'):
    row_list = []
    for cell in row.find_all('td'):
        #print(cell.text.strip())
        row_list.append(cell.text.strip())
    
    #Make sure to ignore entries that haven't got a borough
    if len(row_list) > 0 and row_list[1] != 'Not assigned':
        #Define "Not assigned" neighbourhoods as their borough name
        if row_list[2] == 'Not assigned':
            row_list[2] = row_list[1]
        
        table_list.append(row_list)

##### Here we are going to treat multiple postcode rows...

In [71]:
loaded_postcodes = []
unique_postcode_table_list = []
remaining_rows = []

#print(len(table_list)) #212

for el in table_list:
    #print(el)
    
    if loaded_postcodes.count(el[0]) == 0: #new item, load normally
        loaded_postcodes.append(el[0]) #increment loaded postcodes
        unique_postcode_table_list.append(el)
    else:
        remaining_rows.append(el)
        

for el in remaining_rows:
    postcode = el[0]
    neighbourhood = el[2]
    
    for unique_el in unique_postcode_table_list:
        if unique_el[0] == postcode:
            unique_postcode_table_list[unique_postcode_table_list.index(unique_el)][2] = unique_postcode_table_list[unique_postcode_table_list.index(unique_el)][2] + ', ' + neighbourhood
            break

#### Now that we have the data, just create a dataframe and populate it

In [74]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
ca_postcodes_df = pd.DataFrame(columns=column_names)


for item in unique_postcode_table_list:
    PostalCode = item[0]
    Borough = item[1]
    Neighborhood = item[2]
    
    ca_postcodes_df = ca_postcodes_df.append({'PostalCode': PostalCode,
                                              'Borough': Borough,
                                              'Neighborhood': Neighborhood}, ignore_index=True)
        
ca_postcodes_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [73]:
ca_postcodes_df.shape

(103, 3)

## Part Two! Read from csv and get latitude/longitude

In [75]:
lat_log_df = pd.read_csv('Geospatial_Coordinates.csv')

In [77]:
lat_log_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Merge both dataframes

In [93]:
lat_log_df_new = lat_log_df.copy()
#lat_log_df_new['Postal Code'] = lat_log_df_new['PostalCode']
lat_log_df_new.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
#lat_log_df_new
df_pc_lat_long = pd.merge(ca_postcodes_df, lat_log_df_new, how='inner', on = 'PostalCode')
df_pc_lat_long.head()
    

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


### Part 3! Explore and cluster the neighborhoods in Toronto

##### Use geopy library to get the latitude and longitude values of Toronto, Toronto

In [108]:
#Use only boroughs that contain "Toronto", for simplicity

toronto_data = df_pc_lat_long[df_pc_lat_long['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park, Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [109]:
address = 'Toronto, Ca'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Toronto are 43.7170226, -79.4197830350134.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [110]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Define Foursquare Credentials and Version

In [111]:
CLIENT_ID = '0SO1PH0KFTKPTIEDJAQGCGY0DR3Y3F24OWHWMOOGHQDB5MFE' # your Foursquare ID
CLIENT_SECRET = 'BKAUUWFXAFI1RIWM35RDUQJB11Q0GGBTR55NOVWIWSPKPQXX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0SO1PH0KFTKPTIEDJAQGCGY0DR3Y3F24OWHWMOOGHQDB5MFE
CLIENT_SECRET:BKAUUWFXAFI1RIWM35RDUQJB11Q0GGBTR55NOVWIWSPKPQXX


#### Let's create a function to get all venues from all the neighborhoods in Toronto in a 500 miles radius

In [114]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT = 100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [115]:
# get the Toronto Venues
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Harbourfront, Regent Park, Regent Park
Ryerson, Garden District, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond, King, Richmond
Dovercourt Village, Dufferin, Dufferin
Harbourfront East, Toronto Islands, Union Station, Toronto Islands, Union Station
Little Portugal, Trinity, Trinity
The Danforth West, Riverdale, Riverdale
Design Exchange, Toronto Dominion Centre, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar, India Bazaar
Commerce Court, Victoria Hotel, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West, Forest Hill West
High Park, The Junction South, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville, North Midtown, Yorkville
Parkdale, Roncesvalles, Roncesvalles
Davisville
Harbord, University of Toronto, University of Toronto
Runnymede, Swansea, Swansea
Moore 

In [117]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park, Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park, Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park, Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront, Regent Park, Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Harbourfront, Regent Park, Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


#### Analyse each neighbourhood

In [118]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [119]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.010000,0.000000,0.000000,0.00,0.010000,0.000000,0.01
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00
2,"Brockton, Exhibition Place, Parkdale Village, ...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00
3,Business reply mail Processing Centre969 Eastern,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.000000,0.000000,0.083333,0.083333,0.083333,0.166667,0.166667,0.166667,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00
5,"Cabbagetown, St. James Town, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00
6,Central Bay Street,0.012500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.012500,0.000000,0.000000,0.00,0.012500,0.000000,0.00
7,"Chinatown, Grange Park, Kensington Market, Gra...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.051020,0.000000,0.051020,0.00,0.010204,0.000000,0.00
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00
9,Church and Wellesley,0.011364,0.011364,0.011364,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.011364,0.011364,0.00,0.000000,0.011364,0.00


#### Creating the dataframe with top 10 venues

In [122]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond, King, Richmond",Coffee Shop,Café,American Restaurant,Steakhouse,Thai Restaurant,Gym,Hotel,Cosmetics Shop,Restaurant,Asian Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Bakery,Steakhouse,Seafood Restaurant,Farmers Market,Café,Cheese Shop,Pub
2,"Brockton, Exhibition Place, Parkdale Village, ...",Coffee Shop,Café,Breakfast Spot,Performing Arts Venue,Gym,Furniture / Home Store,Pet Store,Climbing Gym,Caribbean Restaurant,Burrito Place
3,Business reply mail Processing Centre969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Smoke Shop,Park,Farmers Market,Fast Food Restaurant,Spa,Brewery,Burrito Place
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Terminal,Airport Service,Airport Lounge,Harbor / Marina,Airport Gate,Sculpture Garden,Boat or Ferry,Airport Food Court,Airport,Donut Shop
5,"Cabbagetown, St. James Town, St. James Town",Coffee Shop,Restaurant,Bakery,Indian Restaurant,Pizza Place,Italian Restaurant,Pub,Café,Gift Shop,Butcher
6,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Bar,Ice Cream Shop,Burger Joint,Sandwich Place,Salad Place,Bubble Tea Shop,Falafel Restaurant
7,"Chinatown, Grange Park, Kensington Market, Gra...",Café,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Mexican Restaurant,Coffee Shop,Dumpling Restaurant,Chinese Restaurant,Dessert Shop
8,Christie,Grocery Store,Café,Park,Convenience Store,Nightclub,Baby Store,Diner,Athletics & Sports,Restaurant,Italian Restaurant
9,Church and Wellesley,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Burger Joint,Restaurant,Mediterranean Restaurant,Bubble Tea Shop,Gastropub,Pub


## Cluster Neighborhoods

In [125]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

#### Create the combined dataframe... at last!

In [126]:
toronto_merged = toronto_data

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront, Regent Park, Regent Park",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Café,Pub,Breakfast Spot,Restaurant,Theater,Mexican Restaurant,Health Food Store
1,M5B,Downtown Toronto,"Ryerson, Garden District, Garden District",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Middle Eastern Restaurant,Italian Restaurant,Restaurant,Plaza,Diner,Pizza Place
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Restaurant,Hotel,Café,Clothing Store,Italian Restaurant,Cocktail Bar,Bakery,Cosmetics Shop,Gastropub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Pub,Asian Restaurant,Women's Store,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Restaurant,Bakery,Steakhouse,Seafood Restaurant,Farmers Market,Café,Cheese Shop,Pub


# Finally, check the clusters!!!

In [129]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

:)